# **Machine Learning**

## Project 1

## Load Mnist dataset

In the data folder there is the dataset of mnist. Mnists consists of  28𝑥28  grayscale images. In total there are 10 training files train0.txt, train1.txt, ..., train9.txt where each rows of train 𝑘 .txt corresponds to an example that belongs to the class  𝑘 .

The testing data follows the same format.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def load_data():
    
    #-------load train files----------------
    x_train = None
    
    y_train = []

    for i in range( 10 ):
        tmp = pd.read_csv( 'mnistdata/train%d.txt' % i, header=None, sep=" " )
        
        hot_vector = [ 1 if j == i else 0 for j in range(0,10) ]
        
        for j in range( tmp.shape[0] ):
            y_train.append( hot_vector )
            
        if i == 0:
            x_train = tmp
        else:
            x_train = pd.concat( [x_train, tmp] )

    train_data = x_train.as_matrix()
    y_train = np.array( y_train )
    
    #--------load test files------------------
    x_test = None
    
    y_test = []

    for i in range( 10 ):
        tmp = pd.read_csv( 'mnistdata/test%d.txt' % i, header=None, sep=" " )
        
        hot_vector = [ 1 if j == i else 0 for j in range(0,10) ]
        
        for j in range( tmp.shape[0] ):
            y_test.append( hot_vector )
            
        if i == 0:
            x_test = tmp
        else:
            x_test = pd.concat( [x_test, tmp] )

    test_data = x_test.as_matrix()
    y_test = np.array( y_test )
    
    return train_data, test_data, y_train, y_test

In [ ]:
X_train, X_test, Y_train, Y_test = load_data()

print("X_train shape = ",X_train.shape)
print("X_test shape = ",X_test.shape)
print("Y_train shape = ",Y_train.shape)
print("Y_test shape = ",Y_test.shape)


## Load CIFAR-10 dataset

In [ ]:
import pickle
import sys
import os
def unpickle(file):
    """load the cifar-10 data"""

    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data


def load_batch(f_path, label_key='labels'):
    """Internal utility for parsing CIFAR data.
    # Arguments
        fpath: path the file to parse.
        label_key: key for label data in the retrieve
            dictionary.
    # Returns
        A tuple `(data, labels)`.
    """
    with open(f_path, 'rb') as f:
        if sys.version_info < (3,):
            d = pickle.load(f)
        else:
            d = pickle.load(f, encoding='bytes')
            # decode utf8
            d_decoded = {}
            for k, v in d.items():
                d_decoded[k.decode('utf8')] = v
            d = d_decoded
    data = d['data']
    labels = d[label_key]

    data = data.reshape(data.shape[0], 3, 32, 32)
    return data, labels


def load_data(path, negatives=False):
    """Loads CIFAR10 dataset.
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """

    num_train_samples = 50000

    x_train_local = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train_local = np.empty((num_train_samples,), dtype='uint8')

    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        (x_train_local[(i - 1) * 10000: i * 10000, :, :, :],
         y_train_local[(i - 1) * 10000: i * 10000]) = load_batch(fpath)

    fpath = os.path.join(path, 'test_batch')
    x_test_local, y_test_local = load_batch(fpath)

    y_train_local = np.reshape(y_train_local, (len(y_train_local), 1))
    y_test_local = np.reshape(y_test_local, (len(y_test_local), 1))

    if negatives:
        x_train_local = x_train_local.transpose(0, 2, 3, 1).astype(np.float32)
        x_test_local = x_test_local.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        x_train_local = np.rollaxis(x_train_local, 1, 4)
        x_test_local = np.rollaxis(x_test_local, 1, 4)

    return (x_train_local, y_train_local), (x_test_local, y_test_local)


In [ ]:
cifar_10_dir = 'cifar-10-batches-py'

(CF10x_train, CF10y_train), (CF10x_test, CF10y_test) = load_data(cifar_10_dir)

print("Train data (x_train): ", CF10x_train.shape)
print("Train labels (y_train): ", CF10y_train.shape)
print("Test data (x_test): ", CF10x_test.shape)
print("Test labels (y_test): ", CF10y_test.shape)


In [ ]:
#reshape CF10x_train and CF10x_test to a 2 dimensional array

CF10x_train = CF10x_train.reshape(50000,3072)
print(CF10x_train.shape)
CF10x_test = CF10x_test.reshape(10000,3072)
print(CF10x_test.shape)


In [ ]:
def one_hot_encode(x):
    """
        argument x: a list of labels
        return one hot encoding matrix 
    """
    encoded = np.zeros((len(x), 10))
    
    for idx, val in enumerate(x):
        encoded[idx][val] = 1
    
    return encoded

CF10y_train = one_hot_encode(CF10y_train)
print(CF10y_train.shape)
CF10y_test = one_hot_encode(CF10y_test)
print(CF10y_test.shape)


## PLOT MNIST Dataset

In [ ]:
#plot 100 images from the training set


n = 100
sqrt_n = int( n**0.5 )
samples = np.random.randint(X_train.shape[0], size=n)

plt.figure( figsize=(11,11) )

cnt = 0
for i in samples:
    cnt += 1
    plt.subplot( sqrt_n, sqrt_n, cnt )
    plt.subplot( sqrt_n, sqrt_n, cnt ).axis('off')
    plt.imshow( X_train[i].reshape(28,28), cmap='gray'  )

plt.show()


## Preprocessing the Mnist dataset

### Normalize 

In [ ]:
X_train = X_train.astype(float)/255
X_test = X_test.astype(float)/255

## Add a column to the dataset

In [ ]:

X_train = np.hstack( (np.ones((X_train.shape[0],1) ), X_train) )
X_test = np.hstack( (np.ones((X_test.shape[0],1) ), X_test) )
print("X_train shape = ",X_train.shape)
print("X_test shape = ",X_test.shape)

## Preprocessing CIFAR-10 dataset
### Normalize 

In [ ]:
CF10x_train = CF10x_train.astype(float)/255
CF10x_test = CF10x_test.astype(float)/255
print("CF10x_train shape = ",CF10x_train.shape)
print("CF10x_test shape = ",CF10x_test.shape)

## Add a column to the dataset

In [ ]:

CF10x_train = np.hstack( (np.ones((CF10x_train.shape[0],1) ), CF10x_train) )
CF10x_test = np.hstack( (np.ones((CF10x_test.shape[0],1) ), CF10x_test) )
print("CF10X_train shape = ",CF10x_train.shape)
print("CF10X_test shape = ",CF10x_test.shape)

## Softmax

In [ ]:
def softmax( x, ax=1 ):
    m = np.max( x, axis=ax, keepdims=True )#max per row
    p = np.exp( x - m )
    return ( p / np.sum(p,axis=ax,keepdims=True) )

## Hidden layer activation functions

In [ ]:
def activation_option(x,i):
    if i==1 :
        return np.log(1+ np.exp(x))
    elif i==2:
        return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    elif i==3:
        return np.cos(x)

## Derivative of activation functions

In [ ]:
def der_active_option(x,i):
    if i==1 :
        return (np.exp(x)/(1 + np.exp(x)))
    elif i==2:
        return (1-(np.square(np.exp(x)-np.exp(-x))/np.square(np.exp(x)+np.exp(-x))))
    elif i==3:
        return (-np.sin(x))

## Compute cost

In [ ]:
def cost_grad_softmax(W1,W2, X, t, lamda , act_func):
    a = X.dot(W1.T)

    max_error = np.max(a, axis=1)
    
    z=activation_option(a,act_func)
    
    y = softmax(z.dot(W2.T))
    
    # Compute the cost function
    # Using the logsumexp for numerical stability
    Ew = np.sum(t * y) - np.sum(max_error) - \
        np.sum(np.log(np.sum(np.exp(y - np.array([max_error, ] * y.shape[1]).T), 1))) - \
        (0.5 * lamda) * np.sum(np.square(W2))

    h_der = der_active_option(a,act_func)
    
    # calculate gradient
    gradEw1 = (((t - y).dot(W2)*h_der).T).dot(X) - lamda *W1
    gradEw2 = (t - y).T.dot(z) - lamda * W2
    
    return Ew, gradEw1,gradEw2

## Train the network

In [ ]:
def ml_softmax_train(t, X, lamda, W1init, W2init, options, batch_size, act_func):
    
    W1 = W1init
    W2 = W2init

    # Maximum number of iteration of gradient ascend
    _iter = options[0]

    # Tolerance
    tol = options[1]

    # Learning rate
    eta = options[2]

    Ewold = -np.inf
    costs = []
    for i in range( 1, _iter+1 ):
        
        #calculate iteration for a speci batch size
        batch_iter = X.shape[0]//batch_size
        
        X_batch_data = np.zeros((batch_size,X.shape[1]))
        Y_batch_data = np.zeros((batch_size,X.shape[1]))
        
        
        for j in range(1,batch_iter+1):
            batch_init = (j-1)*batch_size
            batch_end = j*batch_size
            X_input = X[batch_init:batch_end,:]
            Y_out = t[batch_init:batch_end,:]
            #print("X_batch data shape in loop : ",X_batch_data.shape)
            
        
            Ew, gradEw1, gradEw2 = cost_grad_softmax(W1, W2, X_input, Y_out, lamda, act_func)
           
            # Update parameters based on gradient ascend
            W1 = W1 + eta * gradEw1
            W2 = W2 + eta * gradEw2
                
            
        # save cost
        costs.append(Ew)
        # Show the current cost function on screen
        print('Iteration : %d, Cost function :%f' % (i, Ew))
        #print("difference = ",np.abs(Ew - Ewold),"\n")

        # Break if you achieve the desired accuracy in the cost function
        if (np.abs(Ew - Ewold) < tol) :
            print("Desired accuracy achieved!")
            print(Ew - Ewold)
            break

        Ewold = Ew

    return W1,W2, costs

##  Weights initialization
### Xavier initialization

In [ ]:
def weights_init(shape0,shape1):
    W = np.random.randn(shape0,shape1)*np.sqrt(2/(shape0+shape1))
    return W

## Gradient checking

In [ ]:
def gradcheck_softmax(W1init,W2init, X, t, lamda, act_func):
    
    W1 = weights_init(*W1init.shape)
    W2 = weights_init(*W2init.shape)
    epsilon = 1e-6
    
    _list = np.random.randint(X.shape[0], size=5)
    x_sample = np.array(X[_list, :])
    t_sample = np.array(t[_list, :])
    
    Ew, gradEw1, gradEw2 = cost_grad_softmax(W1,W2, x_sample, t_sample, lamda, act_func)
    
    numericalGrad1 = np.zeros(gradEw1.shape)
    numericalGrad2 = np.zeros(gradEw2.shape)
    
    # Compute all numerical gradient estimates for W1 
    # the matrix numericalGrad1
    for k in range(numericalGrad1.shape[0]):
        for d in range(numericalGrad1.shape[1]):
            
            #add epsilon to the w[k,d]
            w_tmp = np.copy(W1)
            w_tmp[k, d] += epsilon
            e_plus, _, _ = cost_grad_softmax( w_tmp,W2, x_sample, t_sample, lamda, act_func)

            #subtract epsilon to the w[k,d]
            w_tmp = np.copy(W1)
            w_tmp[k, d] -= epsilon
            e_minus, _, _ = cost_grad_softmax( w_tmp,W2, x_sample, t_sample, lamda, act_func)
            
            #approximate gradient ( E[ w[k,d] + theta ] - E[ w[k,d] - theta ] ) / 2*e
            numericalGrad1[k, d] = (e_plus - e_minus) / (2 * epsilon)
    

    # Compute all numerical gradient estimates for W2
    # the matrix numericalGrad2
    for k in range(numericalGrad2.shape[0]):
        for d in range(numericalGrad2.shape[1]):
            
            #add epsilon to the w[k,d]
            w2_tmp = np.copy(W2)
            w2_tmp[k, d] += epsilon
            e_plus, _, _ = cost_grad_softmax( W1,w2_tmp, x_sample, t_sample, lamda, act_func)

            #subtract epsilon to the w[k,d]
            w2_tmp = np.copy(W2)
            w2_tmp[k, d] -= epsilon
            e_minus, _, _ = cost_grad_softmax( W1, w2_tmp, x_sample, t_sample, lamda, act_func)
            
            #approximate gradient ( E[ w[k,d] + theta ] - E[ w[k,d] - theta ] ) / 2*e
            numericalGrad2[k, d] = (e_plus - e_minus) / (2 * epsilon)
            
    return  gradEw1,gradEw2, numericalGrad1, numericalGrad2 

## Training with Mnist dataset

In [ ]:
#shape of X (N,D)
N, D = X_train.shape

#batch size
batch_size=200

#output units
K = 10

#hidden layer units
M_units =100

# initialization of weights with Xavier
W1init = weights_init(M_units, D)
W2init = weights_init(K, M_units)



# regularization parameter
lamda = 0.1

#activation function number 1,2 or 3
activ_num =3;


# options for gradient descent
#options: Epochs, Tolerance, Learning rate

options = [200, 1e-6, 0.5/N]

# Train the model
W1,W2, costs = ml_softmax_train(Y_train, X_train, lamda, W1init, W2init, options,batch_size, activ_num)



In [ ]:
#Grandient check of model
gradEw1, gradEw2, numericalGrad1, numericalGrad2 = gradcheck_softmax(W1init, W2init, X_train, Y_train, lamda, activ_num)

# Absolute norm
#print( "The difference estimate for gradient of w1 is : ", np.max(np.abs(gradEw1 - numericalGrad1)) )
#print( "The difference estimate for gradient of w2 is : ", np.max(np.abs(gradEw2- numericalGrad2)) )


## Training with CIFAR-10 dataset

In [ ]:
# N of X
N, D = CF10x_train.shape
batch_size=300

K = 10
M_units =100

# initialize w for the gradient ascent
W1init = weights_init(M_units, D)
W2init = weights_init(K, M_units)



# regularization parameter
lamda = 0.1

#activation function number
activ_num =3;

# options for gradient descent
#options: Epochs, Tolerance, Learning rate
options = [100, 1e-6, 0.5/N]

# Train the model
W1,W2, costs = ml_softmax_train(CF10y_train, CF10x_train, lamda, W1init, W2init, options,batch_size, activ_num)

In [ ]:
#Grandient check of model with CF10 dataset
gradEw1, gradEw2, numericalGrad1, numericalGrad2 = gradcheck_softmax(W1init, W2init, CF10x_train, CF10y_train, lamda, activ_num)

# Absolute norm
#print( "The difference estimate for gradient of w1 is : ", np.max(np.abs(gradEw1 - numericalGrad1)) )
#print( "The difference estimate for gradient of w2 is : ", np.max(np.abs(gradEw2- numericalGrad2)) )

## Predict 

In [ ]:
def predict_output(W1,W2,x_test, act_func):
    a = x_test.dot(W1.T)
    
    z=activation_option(a,act_func)
    
    y_pred = softmax(z.dot(W2.T))
    
    y_pred = np.argmax(y_pred,1)
    return y_pred   

## Accuracy

In [ ]:
def accuracy(y_pred,Y_test):
    print("Accuracy : ",np.mean( y_pred == np.argmax(Y_test,1)))

## Predict with Mnist Data

In [ ]:
y_pred = predict_output(W1,W2,X_test, activ_num)
print("y_pred.shape =",y_pred.shape)

accuracy(y_pred,Y_test)


## Predict with CF10 Data

In [ ]:
CF10y_pred = predict_output(W1,W2,CF10x_test, activ_num)
print("y_pred.shape =",CF10y_pred.shape)

accuracy(CF10y_pred,CF10y_test)